In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch
!pip install transformers
!pip install transformers[torch]
!pip install accelerate -U
!pip install datasets
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import tensorflow as tf
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForLanguageModeling, AutoModelForCausalLM, TrainingArguments, Trainer, TextDataset
import math
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import tensorflow as tf
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

In [ ]:
# fileName = '/content/drive/MyDrive/deep/Urdu Tweets Dataset.xlsx'
# # dataset = pd.read_table(fileName)
# DataLoader(filename, batch_size=6, shuffle=False, sampler=None,
#            batch_sampler=None, num_workers=0, collate_fn=None,
#            pin_memory=False, drop_last=False, timeout=0,
#            worker_init_fn=None, *, prefetch_factor=2,
#            persistent_workers=False)
# df = pd.read_excel(fileName)
# df.head(10)

import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
import torch

# Custom dataset class
class MyDataset(Dataset):
    def __init__(self, file_path, max_length=256):
        # Load the data from the Excel file using pandas
        self.data = pd.read_excel(file_path)
        self.tokenizer = AutoTokenizer.from_pretrained("hadidev/gpt2-urdu-smallest")
        self.max_length = max_length
        # Add any necessary data preprocessing steps here

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Implement how to get an item (a row in this case) from the dataset
        row = self.data.iloc[idx]
        text = str(row['Text'])  # Ensure text is treated as a string
        emotions = row['Emotions']

        # Tokenize the text
        encoding = self.tokenizer(text, return_tensors='pt', truncation=True, padding='max_length', max_length=self.max_length)

        # Convert emotions to numerical representation
        # For simplicity, I'm using the length of the emotions list as a numerical representation
        numerical_emotions = len(eval(emotions))

        sample = {'input_ids': encoding['input_ids'].squeeze(),
                  'attention_mask': encoding['attention_mask'].squeeze(),
                  'numerical_emotions': numerical_emotions}

        return sample

# Instantiate the dataset
dataset = MyDataset('/content/drive/MyDrive/deep/Urdu Tweets Dataset.xlsx')

# Create a DataLoader
batch_size = 32
shuffle = True

data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

# Example usage in a training loop
for batch in data_loader:
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    numerical_emotions = batch['numerical_emotions']

    # Your training code here
    # print(input_ids.shape, attention_mask.shape, numerical_emotions)




In [ ]:
print(batch)

{'input_ids': tensor([[64637,   102,    95,  ...,     1,     1,     1],
        [    8,  3072,  3357,  ...,     1,     1,     1],
        [  109,    56,    86,  ...,     1,     1,     1],
        ...,
        [  677,   168,   441,  ...,     1,     1,     1],
        [ 1094,  1073,  2466,  ...,     1,     1,     1],
        [ 9348,   142,    70,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'numerical_emotions': tensor([1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1,
        1, 1, 4, 1, 1, 2, 1, 1])}


In [ ]:
# dataset = load_dataset("mwz/UrduQuotes", split="train")
# dataset = DataLoader(dataset, batch_size = 64, shuffle=True, sampler=None,
#            batch_sampler=None, num_workers=0, collate_fn=None,
#            pin_memory=False, drop_last=False, timeout=0,
#            worker_init_fn=None, *, prefetch_factor=2,
#            persistent_workers=False)

In [ ]:
# dataset = dataset.train_test_split(test_size=0.2)
# dataset["train"][0]

In [ ]:
# dataset = dataset.flatten()
# dataset["train"][0]

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("hadidev/gpt2-urdu-smallest")

In [ ]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["quotes"]])

In [ ]:
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=dataset["train"].column_names,
)

In [ ]:
block_size = 128
def group_texts(sentence):
    # Concatenate all texts.
    concatenated_sentences = {k: sum(sentence[k], []) for k in sentence.keys()}
    total_length = len(concatenated_sentences[list(sentence.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_sentences.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_dataset = tokenized_dataset.map(group_texts, batched=True, num_proc=4)

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
model = AutoModelForCausalLM.from_pretrained("hadidev/gpt2-urdu-smallest", from_tf=True)

In [ ]:
training_args = TrainingArguments(
    output_dir="imdb-clm-model",
    # overwrite_output_dir = True,
    num_train_epochs = 1,
    save_strategy="epoch",
    # eval_delay=30000,
    evaluation_strategy="epoch",
    save_total_limit=3,
    max_steps=1000,
    load_best_model_at_end=True,
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

In [ ]:
model_save_name = 'urdu-textgen-gpt2-tweets.pt'
path = F"/content/drive/My Drive/urdu-textgen-gpt2-tweets.pt"
torch.save(model.state_dict(), path)

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
prompt = "اس"

In [ ]:
model = AutoModelForCausalLM.from_pretrained("hadidev/gpt2-urdu-smallest", from_tf=True)
model_save_name = 'urdu-textgen-gpt2-tweets.pt'
path = F"/content/drive/My Drive/urdu-textgen-gpt2-tweets.pt"
model.load_state_dict(torch.load(path))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("hadidev/gpt2-urdu-smallest")
inputs = tokenizer(prompt, return_tensors="pt").input_ids

In [ ]:
outputs = model.generate(inputs, max_new_tokens=50, do_sample=True, top_k=50, top_p=0.95)

In [ ]:
outputs

In [ ]:
tokenizer.batch_decode(outputs, skip_special_tokens=True)